# Symbols

Symbols are a primitive in Javascript. The only way to create a symbol is via the class constructor `Symbol`, which creates a unique symbol every time it is invoked.

In [6]:
var a = Symbol()
var b = Symbol()

console.log(`a: ${a.toString()}`)
console.log(`a == b: ${a == b}`)

a: Symbol()
a == b: false


undefined

Unlike other primitives, there is no wrapper type for symbols. This was to avoid confusion, since the constructor creates a unique symbol every time.

In [7]:
new Symbol()

TypeError: Symbol is not a constructor

The only intended purpose of a symbol is to be used as an object's property.

In [8]:
obj = {}
obj["provider_id"] = 100
obj[Symbol()] = "secret property?"

'secret property?'

In [9]:
Object.getOwnPropertyNames(obj)

[ 'provider_id' ]

In [10]:
obj.provider_id

100

It's not possible to access the secret property by creating a symbol, since the new symbol will not equal the symbol used as the key.

In [11]:
obj[Symbol()]

undefined

However, we can still get to it with a little bit of reflection.

In [12]:
Object.getOwnPropertySymbols(obj)

[ Symbol() ]

In [13]:
secret_key = Object.getOwnPropertySymbols(obj)[0]
obj[secret_key]

'secret property?'

Let's use this approach to see how we could get "private" methods.

In [24]:
"use strict"
var validLocations = Symbol()
var Locations = class {
    constructor(locations) {
        this.locations = locations
    }
       
    getIds() {
        locs = this[validLocations]()
        return locs.map((loc) => {return loc.id})
    }
    
    [validLocations]() {
        return this.locations.filter((loc) => {return loc.id != undefined && loc.id != null})
    }
}
var c = new Locations([{id: 1}, {id: null}, {id: 10}])

'use strict'

In [25]:
c.getIds()

[ 1, 10 ]

In [17]:
Object.getOwnPropertyNames(c.__proto__)

[ 'constructor', 'getIds' ]

In [26]:
Object.getOwnPropertySymbols(c.__proto__)

[ Symbol() ]

In [30]:
var private_method = Object.getOwnPropertySymbols(c.__proto__)[0]
c[private_method]()

[ { id: 1 }, { id: 10 } ]

## Enums?
The semantics are cumbersome, but Symbols can also be used to implement a basic enum.

In [14]:
'use strict'
var Suits = {
    HEARTS: Symbol("Hearts"),
    DIAMONDS: Symbol("Diamonds"),
    CLUBS: Symbol("Clubs"),
    SPADES: Symbol("Spades"),
    toString: (suit) => {
        // There is no method for retrieving a string from a symbol. Here's a workaround that leverages toString()
        var pattern = /^Symbol\((.+)\)$/
        return suit.toString().match(pattern)[1]
    }
}

var Card = class {
    constructor(rank, suit) {
        this.rank = String(rank)
        this.suit = suit
    }
    
    toString() {
        return `${this.rank} of ${Suits.toString(this.suit)}`
    }
}

'use strict'

In [77]:
c = new Card(1, Suits.HEARTS)
c.toString()

'1 of Hearts'

## Well Known Symbols
Well-known symbols are special symbols exposed by javascript for developers to implement. They are similar to Python's dunder ("magic") methods.
Let's look at the well-known symbol `Symbol.iterator`, which can be thought of as an analog to Python's `__iter__`

In [90]:
'use strict'
var Deck = class {
    constructor() {
        this.cards = []
        for (let suit of [Suits.HEARTS, Suits.DIAMONDS, Suits.SPADES, Suits.CLUBS]) {
            for (let i=1;i<14;i++) {
                this.cards.push(new Card(i, suit))
            }
        }
    }
    
    [Symbol.iterator]() {
        return this.cards[Symbol.iterator]()
    }
}

'use strict'

In [99]:
'use strict'
var deck = new Deck()
for (let card of deck) {
    if (card.suit == Suits.SPADES) {
        console.log(card.toString())
    }
}

1 of Spades
2 of Spades
3 of Spades
4 of Spades
5 of Spades
6 of Spades
7 of Spades
8 of Spades
9 of Spades
10 of Spades
11 of Spades
12 of Spades
13 of Spades


undefined